# Nonlinear minimization problems
We consider problems of the form

$$\text{find } u \in V \text{ s.t. } E(u) \leq E(v) \quad \forall~  v \in V.$$

In [ ]:
import netgen.gui
%gui tk
import tkinter
from ngsolve import *

## Scalar minimization problems
As a first example we take $V = H^1_0$ and 
$$
E(u) = \int_{\Omega} \vert \nabla u \vert^2 + u^4 - fu ~dx.
$$
The minimization is equivalent to solving the nonlinear PDE:
$$  - \Delta u + 4 u^3 = f \text{ in } \Omega $$
We solve the PDE with a Newton iteration.

In [ ]:
from netgen.geom2d import unit_square

mesh = Mesh (unit_square.GenerateMesh(maxh=0.2))
V = H1(mesh, order=4, dirichlet=[1,2,3,4])
u = V.TrialFunction()

To solve the problem we use the "SymbolicEnergy" integrator. Based on the symbolic description of the energy functional, it is able to 
* evaluate the energy functional ("Energy")
$$E(u) \qquad (E:V \to \mathbb{R})$$
* compute the Gateau derivative for a given $u$ ("Apply"):
$$A(u)(v) = E'(u)(v) \qquad (A(u): V \to \mathbb{R})$$
* compute the second derivative ("AssembleLinearization")
$$(\delta A)(w)(u,v) \qquad (\delta A(w): V\times V \to \mathbb{R})$$


In [ ]:
a = BilinearForm (V, symmetric=False)
a += SymbolicEnergy ( grad(u)*grad(u) + u*u*u*u-u )

We recall the Newton iteration (cf. [unit-3.5](../unit-3.5-nonlinear/nonlinear.ipynb) ) we make the loop:
* Given an initial guess $u^0$
* loop over $i=0,..$ until convergence:
  * Compute linearization: $A u^i + \delta A(u^i) \Delta u^{i} = 0 $:
    * $f^i = A u^i$ 
    * $B^i = \delta A(u^i)$ 
    * Solve $B^i \Delta u^i = -f^i$
  * Update $u^{i+1} = u^i + \Delta u^{i}$
  * Evaluate stopping criteria
* Evaluate $E(u^{i+1})$

As a stopping criteria we take $ \langle A u^i,\Delta u^i \rangle = \langle A u^i, A u^i \rangle_{(B^i)^{-1}}< \varepsilon$.  

In [ ]:
def SolveNonlinearMinProblem(a,gfu):
    res = gfu.vec.CreateVector()
    du  = gfu.vec.CreateVector()

    for it in range(12):
        print ("Newton iteration", it)
        print ("energy = ", a.Energy(gfu.vec))
    
        #solve linearized problem:
        a.Apply (gfu.vec, res)
        a.AssembleLinearization (gfu.vec)
        inv = a.mat.Inverse(V.FreeDofs())
        du.data = inv * res
    

        #update iteration
        gfu.vec.data -= du

        #stopping criteria
        stopcritval = sqrt(abs(InnerProduct(du,res)))
        print ("<A u",it,", A u",it,">_{-1}^0.5 = ", stopcritval)
        if stopcritval < 1e-13:
            break
        Redraw(blocking=True)

In [ ]:
gfu = GridFunction (V)
gfu.vec[:] = 0
Draw(gfu,mesh,"u")

SolveNonlinearMinProblem(a,gfu)

print ("energy = ", a.Energy(gfu.vec))
    

## Nonlinear elasticity

We consider a beam which is fixed on one side and is subject to gravity only. We assume a Neo-Hookean hyperelastic material. The model is a nonlinear minimization problem with 
$$
  E(v) := \int_{\Omega} \frac{\mu}{2} ( \operatorname{tr}(F^T F-I)+\frac{2 \mu}{\lambda} \operatorname{det}(F^T F)^{\frac{\lambda}{2\mu}} - 1) - \gamma ~ (f,v) ~~ dx
$$
where $\mu$ and $\lambda$ are the Lamé parameters and $F = I + D u$ where $v: \Omega \to \mathbb{R}^2$ is the sought for displacement.

In [ ]:
import netgen.geom2d as geom2d
from ngsolve import *

geo = geom2d.SplineGeometry()
pnums = [ geo.AddPoint (x,y,maxh=0.01) for x,y in [(0,0), (1,0), (1,0.1), (0,0.1)] ]
for p1,p2,bc in [(0,1,"bot"), (1,2,"right"), (2,3,"top"), (3,0,"left")]:
     geo.Append(["line", pnums[p1], pnums[p2]], bc=bc)
mesh = Mesh(geo.GenerateMesh(maxh=0.05))

# E module and poisson number:
E, nu = 210, 0.2
# Lamé constants:
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

V = H1(mesh, order=2, dirichlet="left", dim=mesh.dim)
u  = V.TrialFunction()

#gravity:
force = CoefficientFunction( (0,-1) )

# some utils:
def IdentityCF(dim):
    return CoefficientFunction( tuple( [1 if i==j else 0 for i in range(dim) for j in range(dim)]), dims=(dim,dim) )

def Trace(mat):
    return sum( [mat[i,i] for i in range(mat.dims[0]) ])

def Det(mat):
    if mat.dims[0] == 2:
        return mat[0,0]*mat[1,1]-mat[0,1]*mat[1,0]

def Pow(a, b):
    return exp (log(a)*b)
    
def NeoHook (C):
    return 0.5 * mu * (Trace(C-I) + 2*mu/lam * Pow(Det(C), -lam/2/mu) - 1)

I = IdentityCF(mesh.dim)
F = I + u.Deriv()   # attention: row .. component, col .. derivative
C = F * F.trans  

factor = Parameter(1.0)

a = BilinearForm(V, symmetric=False)
a += SymbolicEnergy(  NeoHook (C).Compile() )
a += SymbolicEnergy(  (-factor * InnerProduct(force,u) ).Compile() )

We want to solve the minimization problem for $\gamma = 5$. Due to the high nonlinearity in the problem, the Newton iteration will not convergence with any initial guess. We approach the case $\gamma = 5$ by solving problems with $\gamma = i/10$ for $i=1,..,50$ and taking the solution of the previous problem as an initial guess.

In [ ]:
gfu = GridFunction(V)
gfu.vec[:] = 0

Draw (gfu, mesh, "u")
SetVisualization (deformation=True)

res = gfu.vec.CreateVector()
du = gfu.vec.CreateVector()

for loadstep in range(50):
    print ("loadstep", loadstep)
    factor.Set ((loadstep+1)/10)
    SolveNonlinearMinProblem(a,gfu)
    Redraw()

## Allen-Cahn equation

The Allen-Cahn equations describe the process of phase separation and is the ($L^2$) gradient-flow equation to the energy
$$
  E(v) = \int_{\Omega} \varepsilon \vert \nabla v \vert^2~+~v^2(1-v^2) ~ dx
$$
i.e. the solution to the Allen-Cahn equation solves
$$
\partial_t u = \frac{\delta E}{\delta u}
$$
The quantity $u$ is an indicator for a phase where $-1$ refers to one phase and $1$ to another phase. 
The equation has two driving forces:
* $u$ is pulled into one of the two minima ($-1$ and $1$) of the nonlinear term $u^2(1-u^2)$ (separation of the phases)
* the diffusion term scaled with $\varepsilon$ enforces a smooth transition between the two phases. $\varepsilon$ determines the size of the transition layer

We use the "SymbolicEnergy" feature to formulate the energy minimization problem and combine it with an implicit Euler discretization:
$$
 M u^{n+1} - M u^n = \Delta t \underbrace{\frac{\delta E}{\delta u}}_{=:A(u)} (u^{n+1})
$$
which we can interpreted as a nonlinear minimization problem again with the energy
$$
  E^{IE}(v) = \int_{\Omega} \frac{\varepsilon}{2} \vert \nabla v \vert^2~+~v^2(1-v^2) + \frac{1}{2\Delta t} \vert v - u^n \vert^2 ~ dx
$$

To solve the nonlinear equation at every time step we again rely on Newton's method.

In [ ]:
from ngsolve import *

from netgen.geom2d import *


periodic = SplineGeometry()
pnts = [ (0,0), (1,0), (1,1), (0,1) ]
pnums = [periodic.AppendPoint(*p) for p in pnts]

lright = periodic.Append ( ["line", pnums[0], pnums[1]],bc="periodic")
btop = periodic.Append ( ["line", pnums[1], pnums[2]], bc="periodic")
periodic.Append ( ["line", pnums[3], pnums[2]], leftdomain=0, rightdomain=1, copy=lright, bc="periodic")
periodic.Append ( ["line", pnums[0], pnums[3]], leftdomain=0, rightdomain=1, copy=btop, bc="periodic")

mesh = Mesh (periodic.GenerateMesh(maxh=0.2))

V = Periodic(H1(mesh, order=4, dirichlet=[]))

u = V.TrialFunction()

eps = 4e-3
dt = 1e-1

gfu = GridFunction(V)
gfuold = GridFunction(V)

a = BilinearForm (V, symmetric=False)
a += SymbolicEnergy (eps/2*grad(u)*grad(u)
                     + ((1-u*u)*(1-u*u))
                     + 0.5/dt*(u-gfuold)*(u-gfuold))

In [ ]:
from math import pi
gfu = GridFunction(V)
gfu.Set(sin(2*pi*x))
#gfu.Set(sin(1e8*x)) #<- essentially a random function
Draw(gfu,mesh,"u")
SetVisualization (deformation=False)
t = 0

In [ ]:
for timestep in range(50):
    gfuold.vec.data = gfu.vec
    SolveNonlinearMinProblem(a,gfu)
    Redraw() 
    t += dt
    print("t = ", t)